In [ ]:
from pathlib import Path
import json
import pandas as pd

# Top transaction Data District

In [57]:
def Top_Transcations_Distrit():

    root_dir = Path("../data/top/transaction/country/india/state/")

    lis = []
    for i in root_dir.rglob("*.json"):
        # print(i)
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)
            
            state = i.parent.parent.name
            year = i.parent.name
            
            quarter = i.stem
            quarter = f'Q{quarter}'

            lis.append({"state": state, "year": year, "quarter": quarter, "data": dataset})



    lis2 = []
    for j in lis:
        for k in j['data']['data']['districts']:
            amount_value = k['metric']['amount']
            amount_formatted = f'{amount_value: .2f}'
            data = dict(state = j['state'],
                        year = j['year'],
                        quarter = j['quarter'],
                        disrict_name = k['entityName'],
                        type = k['metric']['type'],
                        count = k['metric']['count'],
                        amount = amount_formatted,
                        timestamp = j['data']['responseTimestamp'])
            lis2.append(data)
    
    return lis2





In [58]:
TopDistrictName =pd.DataFrame(Top_Transcations_Distrit())
TopDistrictName

,state,year,quarter,disrict_name,type,count,amount,timestamp
0,andaman-&-nicobar-islands,2018,Q1,south andaman,TOTAL,5688,12560249.34,1630501486876
1,andaman-&-nicobar-islands,2018,Q1,nicobars,TOTAL,528,1139848.80,1630501486876
2,andaman-&-nicobar-islands,2018,Q1,north and middle andaman,TOTAL,442,931663.08,1630501486876
3,andaman-&-nicobar-islands,2018,Q2,south andaman,TOTAL,9395,23948235.52,1630501487541
4,andaman-&-nicobar-islands,2018,Q2,nicobars,TOTAL,1120,3072436.91,1630501487541
...,...,...,...,...,...,...,...,...
6507,west-bengal,2023,Q2,hooghly,TOTAL,27753013,52098664362.00,1692619257925
6508,west-bengal,2023,Q2,howrah,TOTAL,27156689,53521241869.00,1692619257925
6509,west-bengal,2023,Q2,paschim medinipur,TOTAL,26646934,49071906041.00,1692619257925
6510,west-bengal,2023,Q2,nadia,TOTAL,26194658,54103857184.00,1692619257925


# Top transaction Data Pincode

In [ ]:
def Top_Transcations_Pincode():

    root_dir = Path("../data/top/transaction/country/india/state/")

    lis = []
    for i in root_dir.rglob("*.json"):
        # print(i)
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)
            
            state = i.parent.parent.name
            year = i.parent.name
            
            quarter = i.stem
            quarter = f'Q{quarter}'

            lis.append({"state": state, "year": year, "quarter": quarter, "data": dataset})

    lis3 = []
    for l in lis:
        for m in l['data']['data']['pincodes']:
            amount_value = m['metric']['amount']
            amount_formatted = f'{amount_value : .2f}'

            data1 = dict(state = l['state'],
                        year = l['year'],
                        quarter = l['quarter'],
                        pincodes = m['entityName'],
                        type = m['metric']['type'],
                        count = m['metric']['count'],
                        amount = amount_formatted,
                        timestamp = l['data']['responseTimestamp'])
            lis3.append(data1)

    return lis3

In [ ]:
TopPincodeName = pd.DataFrame(Top_Transcations_Pincode())
TopPincodeName

# Top Users Data District

In [ ]:
def Top_Users_District():

    root_dir = Path("../data/top/user/country/india/state")

    lis = []
    for i in root_dir.rglob("*.json"):
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)

            state = i.parent.parent.name
            year = i.parent.name

            quarter = i.stem
            quarter = f'Q{quarter}'

            lis.append({"state":state, "year":year, "quarter":quarter, 'data': dataset})

    # print(lis)

    lis2 = []
    for j in lis:
        for k in j['data']['data']['districts']:
            data = dict(state = j['state'],
                        year = j['year'],
                        quarter = j['quarter'],
                        district_name = k['name'],
                        registeredUsers = k['registeredUsers'],
                        timestamp = j['data']['responseTimestamp'])
            lis2.append(data)

    return lis2

In [ ]:
TopUsersDistrict = pd.DataFrame(Top_Users_District())
TopUsersDistrict

# Top Users Data Pincode

In [ ]:
def Top_Users_Pincode():

    root_dir = Path("../data/top/user/country/india/state")

    lis = []
    for i in root_dir.rglob("*.json"):
        with open(i, 'r') as file:
            content = file.read()
            dataset = json.loads(content)

            state = i.parent.parent.name
            year = i.parent.name

            quarter = i.stem
            quarter = f'Q{quarter}'

            lis.append({"state":state, "year":year, "quarter":quarter, 'data': dataset})

    # print(lis)

    lis3 = []
    for l in lis:
        for m in j['data']['data']['pincodes']:
            data = dict(state = l['state'],
                        year = l['year'],
                        quarter = l['quarter'],
                        pincodes = m['name'],
                        registeredUsers = m['registeredUsers'],
                        timestamp = l['data']['responseTimestamp'])
            lis3.append(data)

    return lis3

In [ ]:
TopUsersPincode = pd.DataFrame(Top_Users_Pincode())
TopUsersPincode.iloc[100:150]